In [11]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [12]:
employee_data = [
    [1,'TONY',24000],
    [2,'STEVE',17000],
    [3,'BRUCE',9000],
    [4,'WANDA',20000]
]

employee_schema = ["EMP_ID", "EMP_NAME", "SALARY"]

df_emp = session.createDataFrame(employee_data, schema=employee_schema)
df_emp.show()

------------------------------------
|"EMP_ID"  |"EMP_NAME"  |"SALARY"  |
------------------------------------
|1         |TONY        |24000     |
|2         |STEVE       |17000     |
|3         |BRUCE       |9000      |
|4         |WANDA       |20000     |
------------------------------------



In [13]:
employee_raw_data = [
    [1,'TONY',25000],
    [3,'BRUCE',10000],
    [5,'PETER',15000]
]

df_emp_raw = session.createDataFrame(employee_raw_data, schema=employee_schema)
df_emp_raw.show()

------------------------------------
|"EMP_ID"  |"EMP_NAME"  |"SALARY"  |
------------------------------------
|1         |TONY        |25000     |
|3         |BRUCE       |10000     |
|5         |PETER       |15000     |
------------------------------------



In [14]:
# Create temporary table for Target DataFrame
df_emp.write.mode("overwrite").save_as_table("tmp_emp", table_type="temp")

In [15]:
# Read the temp table as Snowpark Table object
df_tmp_emp = session.table("tmp_emp")

In [17]:
# Merge contents of source DataFrame into temporary table
from snowflake.snowpark.functions import when_matched, when_not_matched

df_tmp_emp.merge(
df_emp_raw,
(df_tmp_emp["EMP_ID"] == df_emp_raw["EMP_ID"]),
[
when_matched().update({
"EMP_NAME": df_emp_raw["EMP_NAME"],
"SALARY": df_emp_raw["SALARY"]
}),
when_not_matched().insert({
"EMP_ID": df_emp_raw["EMP_ID"],
"EMP_NAME": df_emp_raw["EMP_NAME"],
"SALARY": df_emp_raw["SALARY"]
})
]
)# output: MergeResult(rows_inserted=1, rows_updated=2, rows_deleted=0)

MergeResult(rows_inserted=1, rows_updated=2, rows_deleted=0)

In [18]:
df_tmp_emp.show()

------------------------------------
|"EMP_ID"  |"EMP_NAME"  |"SALARY"  |
------------------------------------
|5         |PETER       |15000     |
|2         |STEVE       |17000     |
|1         |TONY        |25000     |
|3         |BRUCE       |10000     |
|4         |WANDA       |20000     |
------------------------------------



In [22]:
from snowflake.snowpark.functions import when_matched, when_not_matched

In [24]:
df_tmp_emp.merge(
df_emp_raw)

TypeError: Table.merge() missing 2 required positional arguments: 'join_expr' and 'clauses'